# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [2]:
import graphlab

A newer version of GraphLab Create (v1.8) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data5b.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] 1453095254 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Python27\lib\site-packages\certifi\cacert.pem
1453095254 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to vthukral@gmail.com and will expire on September 26, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-18776 - Server binary: C:\Python27\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\vaibhavt\AppData\Local\Temp\graphlab_server_1453095254.log.0
[INFO] GraphLab Server Version: 1.7.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [4]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]        

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [9]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [10]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)

To test the function, run the following:

In [11]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
print simple_feature_matrix
print simple_feature_matrix[:,0]
print output

[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]
 [  1.00000000e+00   7.70000000e+02   2.00000000e+00]
 ..., 
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]
 [  1.00000000e+00   1.60000000e+03   3.00000000e+00]
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]]
[ 1.  1.  1. ...,  1.  1.  1.]
[ 221900.  538000.  180000. ...,  402101.  400000.  325000.]


Don't forget to normalize features:

In [13]:
(norm_feature_matrix, norms) = normalize_features(simple_feature_matrix)
print norm_feature_matrix
print norms

[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


We assign some random set of initial weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [15]:
prediction = predict_output(norm_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [16]:
feature_0 = norm_feature_matrix[:,0]
right = output - prediction + weights[0]*feature_0
ro_0 = np.dot(feature_0, right)

feature_1 = norm_feature_matrix[:,1]
right = output - prediction + weights[1]*feature_1
ro_1 = np.dot(feature_1, right)

feature_2 = norm_feature_matrix[:,2]
right = output - prediction + weights[2]*feature_2
ro_2 = np.dot(feature_2, right)

print ro_0
print ro_1
print ro_2

79400300.0349
87939470.773
80966698.676


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [17]:
print ro_1*2
print ro_2*2

175878941.546
161933397.352


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [18]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    left = feature_matrix[:,i]
    right = (output - prediction + weights[i]*left)
    ro_i = np.dot(left, right)

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [19]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [20]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    iterCount = 0
    while True:
        max_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change_i = abs(weights[i] - old_weights_i)
            #print change_i
            if (change_i > max_change):
                max_change = change_i
        if (max_change < tolerance):
            break
        print 'iter_count = ', iterCount, ':max_change = ', max_change
        iterCount = iterCount + 1
            
    return weights


Using the following parameters, learn the weights on the sales dataset. 

In [21]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [22]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [23]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

predicted_output = predict_output(normalized_simple_feature_matrix, weights)
print weights
print predicted_output

residuals = predicted_output - output
residuals_sq = residuals*residuals
rss = residuals_sq.sum()
print rss

iter_count =  0 :max_change =  79400304.6581
iter_count =  1 :max_change =  9138168.3665
iter_count =  2 :max_change =  8194809.40937
iter_count =  3 :max_change =  6598905.03827
iter_count =  4 :max_change =  5522173.1943
iter_count =  5 :max_change =  4621129.81031
iter_count =  6 :max_change =  3867108.10625
iter_count =  7 :max_change =  3236118.81061
iter_count =  8 :max_change =  2708086.93955
iter_count =  9 :max_change =  2266213.1094
iter_count =  10 :max_change =  1896439.06265
iter_count =  11 :max_change =  1587000.40319
iter_count =  12 :max_change =  1328052.31095
iter_count =  13 :max_change =  1111356.328
iter_count =  14 :max_change =  930018.251241
iter_count =  15 :max_change =  778268.79269
iter_count =  16 :max_change =  651280.029038
iter_count =  17 :max_change =  545011.800818
iter_count =  18 :max_change =  456083.174344
iter_count =  19 :max_change =  381664.876995
iter_count =  20 :max_change =  319389.283634
iter_count =  21 :max_change =  267275.090397
iter

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [24]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [25]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [26]:
(train_feature_matrix, train_output) = get_numpy_data(train_data, all_features, 'price')
(normalized_train_feature_matrix, train_norms) = normalize_features(train_feature_matrix) # normalize features
print train_norms

[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.04107815e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [27]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e7

iter_count =  0 :max_change =  71114625.7528
iter_count =  1 :max_change =  5024356.36869
iter_count =  2 :max_change =  5012324.91395
iter_count =  3 :max_change =  5000676.42086
iter_count =  4 :max_change =  4991243.17314
iter_count =  5 :max_change =  4984361.34335
iter_count =  6 :max_change =  4979504.92921
iter_count =  7 :max_change =  4973806.56525
iter_count =  8 :max_change =  2879222.99938
iter_count =  9 :max_change =  2706150.76622
iter_count =  10 :max_change =  2532295.64864
iter_count =  11 :max_change =  2368316.57782
iter_count =  12 :max_change =  2213124.35067
iter_count =  13 :max_change =  1552443.60013
iter_count =  14 :max_change =  507838.033159
iter_count =  15 :max_change =  438814.370591
iter_count =  16 :max_change =  371479.622608
iter_count =  17 :max_change =  313415.105581
iter_count =  18 :max_change =  264244.832139
iter_count =  19 :max_change =  222753.8608
iter_count =  20 :max_change =  187770.667306
iter_count =  21 :max_change =  158280.093357


***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [28]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e8

iter_count =  0 :max_change =  71114625.7528
[ 71114625.75280938         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [29]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_feature_matrix, train_output,
                                            initial_weights, l1_penalty, tolerance)
print weights1e4

iter_count =  0 :max_change =  71114625.7528
iter_count =  1 :max_change =  8464015.94738
iter_count =  2 :max_change =  6818451.35869
iter_count =  3 :max_change =  5570165.21901
iter_count =  4 :max_change =  4559838.46759
iter_count =  5 :max_change =  3732056.69715
iter_count =  6 :max_change =  3053211.61353
iter_count =  7 :max_change =  2497517.43558
iter_count =  8 :max_change =  2043868.97325
iter_count =  9 :max_change =  1674717.70906
iter_count =  10 :max_change =  1375420.75692
iter_count =  11 :max_change =  1296621.8143
iter_count =  12 :max_change =  1257340.9361
iter_count =  13 :max_change =  1228271.24819
iter_count =  14 :max_change =  1206854.36471
iter_count =  15 :max_change =  1191059.40598
iter_count =  16 :max_change =  1179030.77017
iter_count =  17 :max_change =  1169777.43345
iter_count =  18 :max_change =  1162340.67589
iter_count =  19 :max_change =  1155980.52901
iter_count =  20 :max_change =  1150139.67966
iter_count =  21 :max_change =  1144406.32752


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [32]:
normalized_weights1e7 = weights1e7 / train_norms
normalized_weights1e8 = weights1e8 / train_norms
normalized_weights1e4 = weights1e4 / train_norms
print normalized_weights1e7[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [34]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [35]:
predicted_test_output = predict_output(test_feature_matrix, normalized_weights1e7)
residuals = predicted_test_output - test_output
residuals_sq = residuals * residuals
rss = residuals_sq.sum()
print rss

2.75962079909e+14


In [36]:
predicted_test_output = predict_output(test_feature_matrix, normalized_weights1e8)
residuals = predicted_test_output - test_output
residuals_sq = residuals * residuals
rss = residuals_sq.sum()
print rss

5.37166150034e+14


In [37]:
predicted_test_output = predict_output(test_feature_matrix, normalized_weights1e4)
residuals = predicted_test_output - test_output
residuals_sq = residuals * residuals
rss = residuals_sq.sum()
print rss

2.2778100476e+14


***QUIZ QUESTION***

Which model performed best on the test data?